In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers sentence-transformers googletrans==4.0.0-rc1
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install qwen-vl-utils[decord]==0.0.8
!pip install --upgrade --no-cache-dir transformers
!pip install --upgrade timm
!pip install --upgrade accelerate
!pip install --upgrade git+https://github.com/huggingface/transformers.git


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=58530f1cae9d3ed0b9819e358972b2e94475468b6c357bea8c2dd7d3eda56987
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

In [2]:
# Import required libraries
import time
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from huggingface_hub import login
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
# Authenticate with Hugging Face (needed for gated models)
login()

# Download NLTK tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Define Student Email (from Assignment Instructions)
email_text = """Subject: Concerns About Professor X’s Conduct

Dear Dr. Ustaad,
I hope this email finds you well. I am writing to express my concerns about Professor X’s conduct 
during the Introduction to Zoology class last semester. On multiple occasions, Professor X made 
dismissive remarks about students’ questions and failed to provide clear feedback on assignments.  

Additionally, the grading seemed inconsistent and unfair, with no opportunity for clarification or 
appeal. 

I found this experience deeply frustrating and demotivating, and I believe it affected my performance 
in the course. I would appreciate it if the department could look into this matter and ensure that 
future students have a more positive and supportive learning environment. 

Thank you for your attention to this matter. 
Sincerely, 
Shaagird"""

In [5]:
import torch

# Automatically determine the best device
device = "cuda" if torch.cuda.is_available() and torch.cuda.mem_get_info(0)[0] > 1e9 else "cpu"

# Load summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=-1)

# Load question answering model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2",device_map="auto")

# For keyword extraction, use a separate model designed for sentence embeddings (e.g., 'all-MiniLM-L6-v2')
sbert_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)  # Consider using the best model for your purpose

# Initialize the Translator
translator = Translator()

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def measure_time(task_function, *args, model_name="Unknown", **kwargs):
    start = time.time()
    result = task_function(*args, **kwargs) 
    end = time.time()
    elapsed_time = end - start
    print(f"{model_name} Execution Time: {elapsed_time:.2f} seconds")
    return result, elapsed_time

execution_times = {}

In [7]:
### **TASK 1: Summarization**
print("\nPerforming Summarization...")
summary_dif, execution_times["Summarization_dif"] = measure_time(
    summarizer, email_text, max_length=50, min_length=20, do_sample=False, model_name="Summarization (dif)"
)

# Extract the summarized text
summarized_text = summary_dif[0]['summary_text'] if isinstance(summary_dif, list) else summary_dif


Performing Summarization...
Summarization (dif) Execution Time: 4.83 seconds


In [8]:
### **TASK 2: Question Answering**
print("\nPerforming Question Answering...")
question = "What was the reason for the student's disappointment with Professor X?"
qa_dif, execution_times["QA_dif"] = measure_time(qa_pipeline, question=question, context=email_text, model_name="Question Answering (dif)")


Performing Question Answering...
Question Answering (dif) Execution Time: 0.44 seconds


In [16]:
pip install keybert


Note: you may need to restart the kernel to use updated packages.


In [18]:
### **TASK 3: Keyword Extraction**
print("\nExtracting Keywords...")

from keybert import KeyBERT
import time

# Initialize KeyBERT model
kw_model = KeyBERT("all-MiniLM-L6-v2")  # Uses a small but powerful embedding model

# Start time tracking
start_time = time.time()

# Extract key phrases (n-grams) with KeyBERT
keywords = kw_model.extract_keywords(
    summarized_text, 
    keyphrase_ngram_range=(1, 3),  # Extract single words & short phrases
    stop_words="english",  # Remove common words
    top_n=10  # Extract top 10 keywords
)

# Extract just the words (not scores)
extracted_keywords = [kw[0] for kw in keywords]

# Record execution time
execution_times["Keyword_Extraction"] = time.time() - start_time


Extracting Keywords...


In [11]:
### **TASK 4: Translation (English → French → English)**
print("\nPerforming Translation...")
french_translation, execution_times["Translation_FR"] = measure_time(translator.translate, email_text, src='en', dest='fr', model_name="Translation to French")
back_translation, execution_times["Translation_EN"] = measure_time(translator.translate, french_translation.text, src='fr', dest='en', model_name="Translation back to English")



Performing Translation...
Translation to French Execution Time: 1.13 seconds
Translation back to English Execution Time: 1.03 seconds


In [19]:
# Print Results
print("\n--- RESULTS ---")
print("\nSummarization:")
print(f"Different Model: {summary_dif[0]['summary_text']}")

print("\nQuestion Answering:")
print(f"Different Model: {qa_dif['answer']}")

print(f"Keyword Extraction Execution Time: {execution_times['Keyword_Extraction']:.2f} seconds")
print(f"Extracted Keywords: {extracted_keywords}")

print("\nTranslation:")
print(f"French Translation: {getattr(french_translation, 'translation_text', french_translation)}")
print(f"Back to English Translation: {getattr(back_translation, 'translation_text', back_translation)}")

# Print Execution Times for Each Task
print("\n--- Execution Times (Seconds) ---")
for task, time_taken in execution_times.items():
    print(f"{task}: {time_taken:.2f} sec")


--- RESULTS ---

Summarization:
Different Model: Students complain about Professor X’s conduct in Introduction to Zoology class. On multiple occasions, Professor X made dismissive remarks about students’ questions and failed to provide clear feedback on assignments.

Question Answering:
Different Model: grading seemed inconsistent and unfair
Keyword Extraction Execution Time: 0.05 seconds
Extracted Keywords: ['students complain professor', 'complain professor conduct', 'professor dismissive remarks', 'dismissive remarks students', 'complain professor', 'professor dismissive', 'conduct introduction zoology', 'professor conduct', 'students complain', 'zoology class']

Translation:
French Translation: Translated(src=en, dest=fr, text=Sujet: Préoccupations concernant la conduite du professeur X

Cher Dr. Ustaad,
J'espère que cet e-mail vous trouve bien.J'écris pour exprimer mes préoccupations concernant la conduite du professeur X
Lors de la classe d'introduction à la zoologie dernier sem